In [1]:
import os

import numpy as np
import cv2
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.losses import Loss
from scipy import ndimage
from sklearn.model_selection import train_test_split

physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [2]:
train_data = np.zeros((110,300,300,3), dtype=np.float32)

for i in range(0, 110):
    img = keras.preprocessing.image.load_img('Datasets/crop_data/' + str(i) + '.png')
    img_arr = keras.preprocessing.image.img_to_array(img)
    
    train_data[i] = img_arr/255
    
data = pd.read_csv('Datasets/labels.csv')
pair1 = data[['m11','m22']][:110].to_numpy()
pair2 = data[['m12','m21']][:110].to_numpy()
pair3 = data[['m31','m32']][:110].to_numpy()

pair1 = pair1.astype(np.float32)
pair2 = pair2.astype(np.float32)
pair3 = pair3.astype(np.float32)

In [2]:
train_data = np.zeros((100,1000,1000,3), dtype=np.float32)

for i in range(0, 100):
    img = keras.preprocessing.image.load_img('Datasets/synthetic_data/' + str(i) + '.png')
    img_arr = keras.preprocessing.image.img_to_array(img)
    
    train_data[i] = img_arr/255

In [3]:
def perspective_correction():
    inputs = keras.Input(shape=(300, 300, 3), batch_size=None)
    
    #dense_net
    x = keras.applications.DenseNet121(include_top=False)(inputs)
    
    x = layers.Conv2D(filters=128, kernel_size=(3,3), padding='same')(x)
    x = layers.MaxPool2D()(x)
    x = layers.Activation('relu')(x)
    
    x = layers.Conv2D(filters=128, kernel_size=(3,3), padding='same')(x)
    x = layers.MaxPool2D()(x)
    x = layers.Activation('relu')(x)
    
#     x = layers.Conv2D(filters=128, kernel_size=(3,3), padding='same')(x)
#     x = layers.MaxPool2D()(x)
#     x = layers.Activation('relu')(x)
    
#     x = layers.Conv2D(filters=128, kernel_size=(3,3), padding='same')(x)
#     x = layers.MaxPool2D()(x)
#     x = layers.Activation('relu')(x)
    
#     x = layers.Conv2D(filters=128, kernel_size=(3,3), padding='same')(x)
#     x = layers.MaxPool2D()(x)
#     x = layers.Activation('relu')(x)
    
#     x = layers.Conv2D(filters=128, kernel_size=(3,3), padding='same')(x)
#     x = layers.MaxPool2D()(x)
#     x = layers.Activation('relu')(x)
    
#     x = layers.Conv2D(filters=128, kernel_size=(3,3), padding='same')(x)
#     x = layers.MaxPool2D()(x)
#     x = layers.Activation('relu')(x)
    
#     x = layers.Conv2D(filters=128, kernel_size=(3,3), padding='same')(x)
#     x = layers.MaxPool2D()(x)
#     x = layers.Activation('relu')(x)
    
    x = layers.Conv2D(filters=128, kernel_size=(2,2), padding='same')(x)
    x = layers.MaxPool2D()(x)
    x = layers.Activation('relu')(x)
    
    x = tf.keras.layers.Flatten()(x)
    
    x = layers.Dense(512)(x)
    x = layers.Dense(256)(x)
    x = layers.Dense(128)(x)
    x = layers.Dense(64)(x)
    x = layers.Dense(32)(x)
    x = layers.Dense(16)(x)
    outputs = layers.Dense(2)(x)
    
    
    model = keras.Model(inputs=inputs, outputs=outputs, name='perspective_correction')
    
    return model

In [4]:
model = perspective_correction()

In [5]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0005), loss=keras.losses.MeanSquaredError(),metrics=['mae', 'mse'])
model.fit(train_data[:100], pair3[:100], epochs=50, batch_size=4)

Epoch 1/50
25/25 [==============================] - 16s 93ms/step - loss: 18.3384 - mae: 1.7514 - mse: 18.3384
Epoch 2/50
25/25 [==============================] - 2s 91ms/step - loss: 0.2410 - mae: 0.3796 - mse: 0.2410
Epoch 3/50
25/25 [==============================] - 2s 91ms/step - loss: 0.0722 - mae: 0.2005 - mse: 0.0722
Epoch 4/50
25/25 [==============================] - 2s 91ms/step - loss: 0.0436 - mae: 0.1677 - mse: 0.0436
Epoch 5/50
25/25 [==============================] - 2s 91ms/step - loss: 0.0220 - mae: 0.1107 - mse: 0.0220
Epoch 6/50
25/25 [==============================] - 2s 92ms/step - loss: 0.0139 - mae: 0.0912 - mse: 0.0139
Epoch 7/50
25/25 [==============================] - 2s 92ms/step - loss: 0.0093 - mae: 0.0734 - mse: 0.0093
Epoch 8/50
25/25 [==============================] - 2s 92ms/step - loss: 0.0071 - mae: 0.0648 - mse: 0.0071
Epoch 9/50
25/25 [==============================] - 2s 92ms/step - loss: 0.0088 - mae: 0.0735 - mse: 0.0088
Epoch 10/50
25/25 [======

In [6]:
model.save('pair3')

INFO:tensorflow:Assets written to: pair3\assets


In [32]:
model.predict(train_data[100:])

array([[0.9860528 , 0.94540364],
       [1.0522157 , 0.8978348 ],
       [0.9210227 , 0.93840843],
       [0.95434827, 0.9766975 ],
       [0.84704536, 0.90418065],
       [0.9946844 , 0.8443483 ],
       [0.7543004 , 0.77132183],
       [1.014823  , 0.99236095],
       [0.9815399 , 0.95446765],
       [1.0461652 , 0.95917267]], dtype=float32)

In [33]:
pair1[100:]

array([[1.1082776 , 0.8640194 ],
       [1.0552411 , 0.802902  ],
       [1.1201586 , 1.0203778 ],
       [1.0106126 , 1.0572326 ],
       [1.0063453 , 1.0524311 ],
       [0.9558921 , 0.89041895],
       [0.84658796, 0.8978612 ],
       [1.0036443 , 0.8890957 ],
       [0.80394083, 0.8479892 ],
       [1.1708386 , 0.9976312 ]], dtype=float32)